In [8]:
import pandas as pd

hrts_in_minutes = {
    'BD': 63,
    'PW': 11.5,
    'DI': 5.5,
    'RO': 3.5,
    'MD': 1.5
}

sampling_interval_minutes = 1

input_filename = '/home/hmd24/project/TOC_Warning_System/data/processed/TOC_dataset_original.csv'
output_filename = '/home/hmd24/project/TOC_Warning_System/data/processed/TOC_dataset_original_shift.csv'

# ==================== 2. 数据处理核心逻辑 (通常无需修改) ====================

print(f"正在读取数据集: {input_filename}")
try:
    df = pd.read_csv(input_filename)
except FileNotFoundError:
    print(f"错误: 未找到文件 '{input_filename}'。请确保文件名正确且文件在同一目录下。")
    exit()

# 创建一个新的DataFrame来存储处理后的数据
# 先复制原始的'Endpoint'列
new_df = df[['timestamp', 'Endpoint']].copy()

print("正在根据HRT进行时序移位...")

# 遍历每个输入特征和其对应的HRT
for feature, hrt in hrts_in_minutes.items():
    if feature not in df.columns:
        print(f"警告: 在数据集中未找到特征 '{feature}'，将跳过此特征。")
        continue

    # 计算需要移动的行数 = HRT / 采样间隔
    # 使用int()确保是整数行
    shift_rows = int(hrt / sampling_interval_minutes)
    
    # 新的列名
    new_col_name = f'{feature}_shifted'

    # 执行移位操作
    # 使用负数是因为我们要将上游的数据“向前”移动，以匹配下游的'Endpoint'
    # 这相当于把过去的数据行移动到当前行，所以是向上移动（负向shift）
    new_df[new_col_name] = df[feature].shift(-shift_rows)
    
    print(f"  - 特征 '{feature}' (HRT={hrt}分钟): 已向上移动 {shift_rows} 行，生成新列 '{new_col_name}'")


# ==================== 3. 清理和导出 ====================

# 移位操作会在数据帧的末尾产生NaN（空值），因为没有未来的数据可以填充
# 这些行对于训练模型是无效的，所以我们需要删除它们
original_rows = len(new_df)
new_df.dropna(inplace=True)
cleaned_rows = len(new_df)

print("\n移位操作在数据末尾产生了NaN值，正在删除这些行...")
print(f"原始数据行数: {original_rows}")
print(f"删除NaN后剩余行数: {cleaned_rows} (减少了 {original_rows - cleaned_rows} 行)")

cols_to_format = [col for col in new_df.columns if col != 'timestamp']
# 对这些列应用.round(2)
new_df[cols_to_format] = new_df[cols_to_format].round(2)
print("格式化完成。")

# 重新排列列的顺序，确保'timestamp'在第一列，'Endpoint'在最后一列
shifted_cols = [f'{feature}_shifted' for feature in hrts_in_minutes.keys() if f'{feature}_shifted' in new_df.columns]
final_cols_order = ['timestamp'] + shifted_cols + ['Endpoint']
new_df = new_df[final_cols_order]

# 导出构建好的新数据集到CSV文件
new_df.to_csv(output_filename, index=False)

print(f"\n处理完成！新的数据集已成功导出到: {output_filename}")

正在读取数据集: /home/hmd24/project/TOC_Warning_System/data/processed/TOC_dataset_original.csv
正在根据HRT进行时序移位...
  - 特征 'BD' (HRT=63分钟): 已向上移动 63 行，生成新列 'BD_shifted'
  - 特征 'PW' (HRT=11.5分钟): 已向上移动 11 行，生成新列 'PW_shifted'
  - 特征 'DI' (HRT=5.5分钟): 已向上移动 5 行，生成新列 'DI_shifted'
  - 特征 'RO' (HRT=3.5分钟): 已向上移动 3 行，生成新列 'RO_shifted'
  - 特征 'MD' (HRT=1.5分钟): 已向上移动 1 行，生成新列 'MD_shifted'

移位操作在数据末尾产生了NaN值，正在删除这些行...
原始数据行数: 10054
删除NaN后剩余行数: 9991 (减少了 63 行)
格式化完成。

处理完成！新的数据集已成功导出到: /home/hmd24/project/TOC_Warning_System/data/processed/TOC_dataset_original_shift.csv
